# Cleaning a shapefile

DATE: 11 June 2020, 18:00 - 21:00 UTC

AUDIENCE: Intermediate

INSTRUCTOR: Martin Bentley, Digital Geoscientist, [Agile](https://agilescientific.com/)

When processing data, we are often not lucky enough to have it perfectly useable immediately. This notebook works through loading, cleaning and saving a shapefile, using `geopandas`, an extension for `pandas` that adds facility for spatial processing.

#### Note
Much of this is standard data cleaning, and does not rely on geopandas per se, except that the data that we want to clean is in a geospatial data format, such as a shapefile. Most of these tools are the same in standard `pandas`, but have been extended in geopandas to work with spatial indices.

This notebook is provided more as an example of data cleaning, which is common when dealing with real data. The result of this notebook can easily be used in whatever GIS software you prefer, since it is a standard shapefile.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

We will start by loading and having a look at the data that we have available.

In [ ]:
fname = '../data/offshore_wells_2011_UTM20_NAD83.shp'

well_data = gpd.read_file(fname)
well_data.head()

Something that we may be interested in is the different companies that have operated in this field. This is equivalent to looking at a column in a spreadsheet.

In [ ]:
well_data['Company']

If we want to get an idea of all the companies present, we can use the `set` function, which returns the unique values from a list-like object:

In [ ]:
set(well_data['Company'])

A number of these companies should probably be consolidated. The most straightforward way to do this is by diving into the dark art of regular expressions. We will make a dictionary of what to look for as the key and what to replace it with as the value.

In [ ]:
replacements = {
    r'\-': '', #  remove '-'
    r' et al': '', #  remove ' et al'
    r' Cda': '', #  remove ' Cda'
    r'EnCana.*$': 'EnCana', #  change 'EnCana' followed by anything to 'EnCana'
    r'PanCanadian(\-|.*)\n.*': 'PanCanadian', #  strip the odd characters after 'PanCanadian'
    r'Mobil.*$': 'Mobil', #  strip anything following 'Mobil'
    r'Shell.*$': 'Shell', #  strip anything following 'Shell'
    r'Exxonmobil': 'ExxonMobil', #  correct capitalisation of 'Exxonmobil' to 'ExxonMobil'
    r'Petocan': 'PetroCan', #  correct spelling
    r'Petrocan': 'PetroCan', #  correct capitalisation of 'Petrocan' to 'PetroCan'
    r'PetroCan*$': 'PetroCan', #  strip anything following 'PetroCan'
    r'^Husky.*\n.*$': 'HBV', #  convert anything starting with 'Husky' to 'HBV' after stripping new line
    r'^Bow Valley.*\n.*$': 'BVH', #  convert anything starting with 'Bow Valley' to 'BVH' after stripping new line
    r'HBV.*$': 'HBV', #  strip anything following 'HBV'
    r'BVH.*$': 'BVH', #  strip anything following 'BVH'
    r'Pex/Tex': 'Pex', #  convert 'Pex/Tex' to 'Pex'
    r'Candian Sup/': 'Canadian Superior', #  correct typo 'Candian Sup/' to 'Canadian Superior'
    r'Canadian Sup\.': 'Canadian Superior', #  expand 'Canadian Sup.' to 'Canadian Superior'
}

In this case, we are going to create a new column (`Owner`) to store the cleaned data, in case we need to retrieve the exact company for some reason. We could change the original GeoDataFrame column by using the `inplace=True` argument to the `replace` method.

In [ ]:
well_data['Owner'] = well_data['Company'].replace(regex=replacements)
set(well_data['Owner'])

## Exercise 1

1. Print a list of the unique values in the `Well_Type` Series.
2. Clean up the `Well_Type` Series to remove the typos and make the data more consistent. We can do this in-place, because the original data does not really give us any additional information. (Hint: look at the `inplace=True` parameter to do this to the original GeoDataFrame.)
    - Change 'Exploratory' to 'Exploration'
    - Change the typo 'Develpoment' to 'Development'
    - Remove the new line, by changing `\n&` to `''`
    - Remove excess whitespace by changing `\s+` to `' '`

In [ ]:
# Print a list of the unique values in the `Well_Type` Series.


In [ ]:
# Print a list of the unique values in the `Well_Type` Series.
set(well_data['Well_Type'])

In [ ]:
# Clean up the `Well_Type` Series to remove typos and make the data more consistent. Do this in-place.
# 
replacements = {
    'key': 'changed_to',
    r'\n&': '',
    r'\s+': ' ',
    'key2': 'changed_to',
}

well_data['Well_Type'].replace(regex=replacements, inplace=True)
well_data['Well_Type']

In [ ]:
# Clean up the `Well_Type` Series to remove typos and make the data more consistent. Do this in-place.
replacements = {
    'Develpoment\/ ': 'Development\/',
    r'\n&': '',
    r'\s+': ' ',
    'Exploratory': 'Exploration',
}

well_data['Well_Type'].replace(regex=replacements, inplace=True)

## Cleaning Column Names

The current column names are not very helpful in some cases, with weird codes and similar. We can probably make these more understandable, and (geo)pandas makes it easy to do so.

#### Note for Shapefile
The maximim length of field names in a shapefile is 10 characters. Some other formats, such as `.gpkg` do not have this limitation.

______________

We can start by getting the current column names.

In [ ]:
print(well_data.columns)

Some of these are clearly cut off due to needing to be less than 10 characters in length, for example `Well_Termi` and `seafl_twt_`. In other cases, we can see that there are duplicates, for example `Well_Name` and `Well_Nam_1`, without any indication of what the difference is. We can do better, even with the limits of the Shapefile format.

We can get a feel for the data with the `head()` method, as used above. The `set()` method is also often helpful to see what different values are in text columns, and may give us a better idea what the data is describing, as we have already seen.

There is a `rename()` method that can take a `dict` of existing Series names as keys and set a new name as the values. We will change the `Well_Termi` to `Well_End` and `Well_Nam_1` to `Well_Code`.

In [ ]:
series_names = {
    'Well_Termi': 'Well_End',
    'Well_Nam_': 'Well_Code',
}
well_data.rename(columns=series_names)

In [ ]:
well_data.head()

As you can see, `well_data` is not changed, since `rename()` returns a copy of the GeoDataFrame. To change it instead of getting a copy, the `inplace=True` option should be added, or the copy assigned to another variable.

In [ ]:
well_data.rename(columns=series_names, inplace=True)
# We can also do this with the following:
# well_data = well_data.rename(columns=series_names)

Now it works!

In [ ]:
print(well_data.columns)
well_data.head()

### Exercise 2

1. What are the different values of `Well_Symb`?
2. What are the different values of `Drilling_U`? In particular, what are the different entries in `Drilling_U` referring to, and what might be a more descriptive name?
3. Change the following column names in the DataFrame:
    * `Total_De_1` to `Dpth_ft`
    * `Total_Dept` to `Dpth_m`
    * `seafl_twt_` to `FloorTWT`
    * `Drilling_U` to something based on the previous answer.

In [ ]:
# What are the different values in `Well_Symb`?


In [ ]:
# What are the different values in `Well_Symb`?
set(well_data['Well_Symb'])

In [ ]:
# What are the different values in `Drilling_U`?


In [ ]:
# What are the different values in `Drilling_U`?
set(well_data['Drilling_U'])

In [ ]:
series_names = {
}

In [ ]:
series_names = {
    'Drilling_U': 'Drill_Ship',
    'Total_De_1': 'Dpth_ft',
    'Total_Dept': 'Dpth_m',
    'Water_Dept': 'Water_Dpth',
    'seafl_twt_': 'FloorTWT',
}

If we are changing a Shapefile, remember that we can not have Series names longer than 10 characters. This will check it for you.

In [ ]:
for series in series_names:
    if len(series) > 10:
        print(f'{series} longer than 10 characters. Will not be able to save as Shapefile.')
    else:
        well_data.rename(columns=series_names, inplace=True)
well_data.columns

In [ ]:
well_data.head()

## Datetimes

If you are familiar with `pandas`, then you will know the utility of `datetime`s. We have some dates in the data, so we should make sure that they are correctly imported if we want to use that for anything involving time series analysis.

#### Note:
It is not possible to write a `datetime` to a Shapefile. If you want to do analysis that uses timeseries, then you may want to save a cleaned dataframe _before_ you convert to `datetime`s. Alternatively, save the GeoDataFrame as a geopackage or similar format that can handle a `datetime`.

In [ ]:
type(well_data['Spud_Date'][0])

As we see, these dates are stored as strings. We can easily convert them to `datetime`s, however. First we will copy our original geodataframe to save later. (If you are doing this conversion with your own data, make sure that you look into the limitations of doing so, if you want to save a shapefile.)

In [ ]:
well_data_original = well_data.copy()
well_data['Spud_Date'] = pd.to_datetime(well_data['Spud_Date'])
type(well_data['Spud_Date'][0])

## Exercise 3

1. Change the `Well_End` Series to `Timestamp`s.
2. Make a Series of the difference in time between the `Spud_Date` and the `Well_End` Series. (Do not add this to our current geodataframe, to make saving it easier later.)
3. What is the biggest difference in days, between the `Spud_Date` and the `Well_End`  Series? (Hint: you may wish to look at the `dt.days` attribute of a `timeDelta`.)

In [ ]:
# Change the `Well_End` Series to `Datetime`s.


In [ ]:
# Change the `Well_End` Series to `Timestamp`s.
well_data['Well_End'] = pd.to_datetime(well_data['Well_End'])

In [ ]:
# Add a Series with the difference in time between the `Spud_Date` and `Well_End` Series.


In [ ]:
# Add a Series with the difference in time between the `Spud_Date` and `Well_End` Series.
time_differences = well_data['Well_End'] - well_data['Spud_Date']

In [ ]:
# What is the biggest time difference, in days, between the `Spud_Date` and `Well_End` Series?


In [ ]:
# What is the biggest time difference, in days, between the `Spud_Date` and `Well_End` Series?
max(time_differences.dt.days)

## Saving files

Once we have made these changes, we would like to save them for future work. Geopandas makes that very easy. Note that we can not write a `datetime` to shapefiles, so we would need to change it (back) to a string if we want to save it. Similarly, if we have a Series of `bool` values (`True` or `False`) we should convert those to `int`s before we save.

First we will see what available options we have to save to. Geopandas uses `fiona` in the background; we will take a look at what that offers us.

In [ ]:
import fiona #  we do not normally need this for saving, it gets used in the background.
fiona.supported_drivers

We can only write to some of these formats: those with `raw` or `rw` tags.

As a format, `gpkg` is becoming more popular, so we will save our geodataframe as that. One nice advantage, not relevant here, is being able to save multiple layers in a single file.

In [ ]:
fname = '../data/cleaned/gpkg-offshore_wells_2011_UTM20_NAD83.gpkg'

well_data.to_file(fname, layer='well_locations', driver='GPKG')

We can also save as a Shapefile, but we will get an error:

`DriverSupportError: ESRI Shapefile does not support datetime fields`

In [ ]:
fname = '../data/cleaned/offshore_wells_2011_UTM20_NAD83_cleaned.shp'

well_data.to_file(fname)

This is fixable by saving our copy of the dataset, or by converting the datetime back to a string.

In [ ]:
# well_data['Spud_Date'] = well_data['Spud_Date'].dt.strftime('%Y-%m-%d')
# well_data['Well_End'] = well_data['Well_End'].dt.strftime('%Y-%m-%d')
fname = '../data/offshore_wells_2011_UTM20_NAD83_cleaned.shp'

well_data_original.to_file(fname)

We can also easily save these using a different CRS, if that is better for our data. This is one for the North America Datum 1927, in degrees.

In [ ]:
fname = '../data/offshore_wells_2011_Geographic_NAD27_cleaned.shp'

well_data_original.to_crs(epsg=4267).to_file(fname, driver='ESRI Shapefile')

## Closing remarks

The data that we have just saved can be used in the "Intro to Geopandas" notebook.

<hr />
<img src="https://avatars1.githubusercontent.com/u/1692321?v=3&s=200" style="float:center" width="40px" />
<p><center>© 2020 <a href="http://www.agilegeoscience.com/">Agile Geoscience</a> — <a href="https://creativecommons.org/licenses/by/4.0/">CC-BY</a></center></p>